In [2]:
import tensorflow as tf

import keras
from keras import layers
from keras.models import Sequential
from keras.layers import Conv2D, Dense, MaxPooling2D, Flatten, Input, BatchNormalization, ReLU


import cv2
import numpy as np
import matplotlib.pyplot as plt

import PIL
import os
from os import listdir

In [3]:
class ResNetBlock(keras.Model):
    def __init__(self, filters, kernel_size = (3,3), strides=1, downsample = False, strides_2 = None):
        super().__init__()
        if strides_2 is None : strides_2 = strides
        self.conv1 = Conv2D(filters, kernel_size, strides = strides, padding = "same")
        self.norm = BatchNormalization()
        self.relu = ReLU()
        self.conv2 = Conv2D(filters, kernel_size, strides = strides_2, padding = "same")
        self.norm2 = BatchNormalization()
        self.downsample = downsample
        if downsample ==True:
            self.downsample_conv = Conv2D(filters, 1, strides)
            self.downsample_norm = BatchNormalization()
        else:
            self.downsample_conv = None
            self.downsample_norm = None
    
    
    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.norm(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.norm2(x)
        x = self.relu(x)
        y = inputs
        if self.downsample:
            y = self.downsample_conv(y)
            y = self.downsample_norm(y)
        #print(x.shape, y.shape)
        x = x + y
        x = self.relu(x)
        return x

In [5]:
inputs = Input((224, 224,3))
x = Conv2D(64, (7,7), strides = 2, padding="same")(inputs)
x = BatchNormalization()(x)
x = ReLU()(x)
x = MaxPooling2D((2,2))(x)


x = ResNetBlock(64)(x) #Block_1
x = ResNetBlock(64)(x) #Block_2
x = ResNetBlock(64)(x) #Block_3

x = ResNetBlock(128, strides = 2, downsample = True, strides_2 = 1)(x) #Block_4

x = ResNetBlock(128)(x) #Block_5
x = ResNetBlock(128)(x) #Block_6
x = ResNetBlock(128)(x) #Block_7

x = ResNetBlock(256, strides = 2, downsample = True, strides_2 = 1)(x) #Block_4

x = ResNetBlock(256)(x) #Block_9
x = ResNetBlock(256)(x) #Block_10
x = ResNetBlock(256)(x) #Block_11
x = ResNetBlock(256)(x) #Block_12
x = ResNetBlock(256)(x) #Block_13

x = ResNetBlock(512, strides = 2, downsample = True, strides_2 = 1)(x) #Block_14

x = ResNetBlock(512)(x) #Block_15
x = ResNetBlock(512)(x) #Block_16

x = layers.GlobalAveragePooling2D()(x)


outputs = Dense(7, activation = "softmax")(x)

model_resnet = keras.Model(inputs=inputs, outputs=outputs)
model_resnet.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_36 (Conv2D)              │ (None, 112, 112, 64)   │         9,472 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_36          │ (None, 112, 112, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_17 (ReLU)                 │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_net_block_16 (ResNetBlock)  │ (None, 56, 56, 64)     │        74,368 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_net_block_17 (ResNetBlock)  │ (None, 56, 56, 64)     │        74,368 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_net_block_18 (ResNetBlock)  │ (None, 56, 56, 64)     │        74,368 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_net_block_19 (ResNetBlock)  │ (None, 28, 28, 128)    │       231,296 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_net_block_20 (ResNetBlock)  │ (None, 28, 28, 128)    │       296,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_net_block_21 (ResNetBlock)  │ (None, 28, 28, 128)    │       296,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_net_block_22 (ResNetBlock)  │ (None, 28, 28, 128)    │       296,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_net_block_23 (ResNetBlock)  │ (None, 14, 14, 256)    │       921,344 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_net_block_24 (ResNetBlock)  │ (None, 14, 14, 256)    │     1,182,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_net_block_25 (ResNetBlock)  │ (None, 14, 14, 256)    │     1,182,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_net_block_26 (ResNetBlock)  │ (None, 14, 14, 256)    │     1,182,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_net_block_27 (ResNetBlock)  │ (None, 14, 14, 256)    │     1,182,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_net_block_28 (ResNetBlock)  │ (None, 14, 14, 256)    │     1,182,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_net_block_29 (ResNetBlock)  │ (None, 7, 7, 512)      │     3,677,696 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_net_block_30 (ResNetBlock)  │ (None, 7, 7, 512)      │     4,723,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_net_block_31 (ResNetBlock)  │ (None, 7, 7, 512)      │     4,723,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 7)              │         3,591 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,313,799 (81.31 MB)

 Trainable params: 21,296,775 (81.24 MB)

 Non-trainable params: 17,024 (66.50 KB)

In [6]:
data_dir = "Data_Cancer/"
labels = listdir(data_dir)
labels

['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']

In [7]:
img_height = 224
img_width = 224
batch_size = 32

##### Creating training, validation dataset

In [8]:
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset = "training",
    seed = 1234,
    image_size = (img_height, img_width),
    batch_size = batch_size
)

validation_dataset = keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset = "validation",
    seed = 1234,
    image_size = (img_height, img_width),
    batch_size = batch_size
)

Found 10015 files belonging to 7 classes.
Using 8012 files for training.
Found 10015 files belonging to 7 classes.
Using 2003 files for validation.


In [9]:
train_dataset = train_dataset.map(lambda x,y: (layers.Rescaling(1./255)(x), y))
validation_dataset = validation_dataset.map(lambda x,y: (layers.Rescaling(1./255)(x), y))

In [11]:
epochs = 10
model_resnet.compile(optimizer = "adam", loss = keras.losses.SparseCategoricalCrossentropy, metrics=["accuracy"])
history = model_resnet.fit(train_dataset, validation_data = validation_dataset, epochs = epochs)

Epoch 1/10


I0000 00:00:1720915250.836146   45772 service.cc:145] XLA service 0x7f6fd8002880 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1720915250.836200   45772 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Laptop GPU, Compute Capability 8.6
2024-07-14 02:00:51.550751: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-14 02:00:53.608919: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907
I0000 00:00:1720915254.851093   46315 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_10303', 8 bytes spill stores, 8 bytes spill loads

I0000 00:00:1720915272.328194   45772 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


250/251 ━━━━━━━━━━━━━━━━━━━━ 0s 247ms/step - accuracy: 0.6297 - loss: 1.3897

2024-07-14 02:02:19.919817: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.13GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


251/251 ━━━━━━━━━━━━━━━━━━━━ 126s 356ms/step - accuracy: 0.6299 - loss: 1.3870 - val_accuracy: 0.3390 - val_loss: 1.5462
Epoch 2/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 67s 266ms/step - accuracy: 0.6766 - loss: 0.9101 - val_accuracy: 0.6855 - val_loss: 0.8914
Epoch 3/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 67s 266ms/step - accuracy: 0.6863 - loss: 0.8696 - val_accuracy: 0.6645 - val_loss: 0.9482
Epoch 4/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 67s 267ms/step - accuracy: 0.6895 - loss: 0.8456 - val_accuracy: 0.4074 - val_loss: 1.5088
Epoch 5/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 67s 267ms/step - accuracy: 0.7028 - loss: 0.8138 - val_accuracy: 0.6410 - val_loss: 0.9397
Epoch 6/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 68s 270ms/step - accuracy: 0.7067 - loss: 0.8067 - val_accuracy: 0.2287 - val_loss: 2.2436
Epoch 7/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 67s 268ms/step - accuracy: 0.7219 - loss: 0.7633 - val_accuracy: 0.6970 - val_loss: 0.7925
Epoch 8/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 31858s 127s/step - accuracy: 0.7304 - loss: 0.7429 - 